In [1]:
import geopandas as gpd

In [2]:
gdf = gpd.read_file("C:/repos/satellite-images-nso/input_data/N2000_pzh.json")

In [3]:
gdf

,naam_n2k,vhn_new,nr,beschermin,sitecode_v,sitecode_h,status,kadaster,staatscour,FID,Code,Provincien,SHAPE_Leng,SHAPE_Area,geometry
0,Biesbosch,3,112,VR+HR,NL3009002,NL3000040,Natura 2000-besluit 4 september 2013,OZ4 63282/153,2013/19978,8,28,Zuid-Holland,6.564802,0.385741,"MULTIPOLYGON (((4.63984 51.72316, 4.63987 51.7..."
1,Boezems Kinderdijk,1,106,VR,NL9802099,None,Natura 2000-besluit 2010,OZ4 59683/109,2011/4458,8,28,Zuid-Holland,6.564802,0.385741,"POLYGON ((4.64741 51.88801, 4.64781 51.88798, ..."
2,"Broekvelden, Vettenbroek & Polder Stein",1,104,VR,NL9802064,None,Natura 2000-besluit 2010,OZ4 59683/108,2011/4458,8,28,Zuid-Holland,6.564802,0.385741,"POLYGON ((4.74340 52.05915, 4.74362 52.05911, ..."
3,Coepelduynen,2,96,HR,None,NL1000030,Natura 2000-besluit 2010,OZ4 57913/192,2009/19769,8,28,Zuid-Holland,6.564802,0.385741,"POLYGON ((4.43428 52.23760, 4.43437 52.23738, ..."
4,De Wilck,1,102,VR,NL9802101,None,Natura 2000-besluit 7 mei 2013,OZ4 62860/123,2013/12211,8,28,Zuid-Holland,6.564802,0.385741,"POLYGON ((4.54282 52.11817, 4.54298 52.11812, ..."
5,Donkse Laagten,1,107,VR,NL9802066,None,Natura 2000-besluit 2010,OZ4 57913/194,2009/19769,8,28,Zuid-Holland,6.564802,0.385741,"POLYGON ((4.77578 51.89279, 4.77618 51.89237, ..."
6,Duinen Goeree & Kwade Hoek,2,101,HR,None,NL9801079,Natura 2000-besluit 2008,OZ4 57922/141,2010/2236,8,28,Zuid-Holland,6.564802,0.385741,"MULTIPOLYGON (((4.01022 51.83393, 4.01109 51.8..."
7,Duinen Goeree & Kwade Hoek,3,101,VR+HR,NL2000006,NL9801079,Natura 2000-besluit 2008,OZ4 57922/141,2010/2236,8,28,Zuid-Holland,6.564802,0.385741,"POLYGON ((4.01022 51.83393, 4.01018 51.83393, ..."
8,Grevelingen,3,115,VR+HR,NL4000021,NL4000021,Natura 2000-besluit 4 september 2013,OZ4 63282/154,2013/19978,8,28,Zuid-Holland,6.564802,0.385741,"MULTIPOLYGON (((4.16017 51.69211, 4.16040 51.6..."
9,Haringvliet,3,109,VR+HR,NL1000015,NL1000015,"Natura 2000 besluit 28 april 2015, Uitvoerings...",OZ4 66296/200,2015/11211,8,28,Zuid-Holland,6.564802,0.385741,"MULTIPOLYGON (((4.39549 51.70893, 4.39543 51.7..."


In [5]:
# Select the natura 2000 region you want to have here along with it's output name.
n2000_name = "Duinen Goeree & Kwade Hoek"
output_name = "Duinen_Goeree_Kwade_Hoek.geojson"

In [6]:
gdf[gdf['naam_n2k'] == n2000_name]

,naam_n2k,vhn_new,nr,beschermin,sitecode_v,sitecode_h,status,kadaster,staatscour,FID,Code,Provincien,SHAPE_Leng,SHAPE_Area,geometry
6,Duinen Goeree & Kwade Hoek,2,101,HR,None,NL9801079,Natura 2000-besluit 2008,OZ4 57922/141,2010/2236,8,28,Zuid-Holland,6.564802,0.385741,"MULTIPOLYGON (((4.01022 51.83393, 4.01109 51.8..."
7,Duinen Goeree & Kwade Hoek,3,101,VR+HR,NL2000006,NL9801079,Natura 2000-besluit 2008,OZ4 57922/141,2010/2236,8,28,Zuid-Holland,6.564802,0.385741,"POLYGON ((4.01022 51.83393, 4.01018 51.83393, ..."


In [9]:
gdf[gdf['naam_n2k'] == n2000_name].to_file(output_name, driver="GeoJSON")

In [29]:
# Check for polygons which are not yet unary_unioned.
if len(gpd.read_file(output_name)) > 1:
    gpd.GeoDataFrame(geometry=[gpd.read_file(output_name).unary_union]).to_file(output_name, driver="GeoJSON")